# Investigation of Grover's algorithm

Based on 
https://pennylane.ai/qml/demos/tutorial_qjit_compile_grovers_algorithm_with_catalyst

Some good sources for the theory behind Grover's algorithm

*  https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/grover-algorithm/grover-algorithm-description

*  

The starting point is searching for x such that $ f(\mid x > ) = - \mid x > $


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml



In [2]:
def equal_superposition(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)


In [3]:
def oracle(wires, omega):
    """
        Apply the Oracle
    """
    qml.FlipSign(omega, wires=wires)



In [11]:
def num_grover_iterations(N, M):
    """
     Set the number of iterations for the Grover's algorithm,
    """
    return np.ceil(np.sqrt(N / M) * np.pi / 4).astype(int)



In [5]:
def grover_circuit(num_qubits):
    """
      Circuit for the Grover's algorithm
    """
    wires = list(range(num_qubits))
    omega = np.array([np.zeros(num_qubits), np.ones(num_qubits)])


    M = len(omega)
    N = 2**num_qubits

    # Initial state preparation
    equal_superposition(wires)

    # Grover iterations
    for _ in range(num_grover_iterations(N, M)):
        for omg in omega:
            oracle(wires, omg)
        qml.templates.GroverOperator(wires)

    return qml.probs(wires=wires)


## Apply the circuit for Grover's algorithm

In [8]:
#NUM_QUBITS = 12
NUM_QUBITS = 6

dev = qml.device("default.qubit", wires=NUM_QUBITS)


@qml.qnode(dev)
def circuit_default_qubit():
    return grover_circuit(NUM_QUBITS)


results = circuit_default_qubit()


In [9]:
def most_probable_states_descending(probs, N):
    """Returns the indices of the N most probable states in descending order."""
    if N > len(probs):
        raise ValueError("N cannot be greater than the length of the probs array.")

    return np.argsort(probs)[-N:][::-1]


def print_most_probable_states_descending(probs, N):
    """Prints the most probable states and their probabilities in descending order."""
    for i in most_probable_states_descending(probs, N):
        print(f"Prob of state '{i:0{NUM_QUBITS}b}': {probs[i]:.4g}")



In [10]:
print_most_probable_states_descending(results, N=4)

Prob of state '000000': 0.9074
Prob of state '100101': 0.001469
Prob of state '100111': 0.001469
Prob of state '100011': 0.001469
